In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os

In [2]:
from tqdm.notebook import tqdm

In [3]:
tqdm.pandas()

In [4]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "train_id": "hierarchical_single",
    'sample_size': 1
})


In [5]:

job_path = "/mnt/disks/data/fma/trains"

# In[15]:

train_path = os.path.join(job_path,args.train_id)

# In[16]:

base_path = os.path.join(args.root_dir,"fma")

# In[17]:

models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")

# In[18]:

metadata_file = os.path.join(train_path,"metadata.json")


categories_labels_path = os.path.join(train_path,"labels.json")


In [6]:


def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp




In [7]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True



In [8]:
create_dir(train_path)

True

## Load genres file. Contains relationships beetwen genres

In [9]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [10]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [11]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


In [52]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'))

In [53]:
tracks_df = tracks_df.sample(frac=args.sample_size)

In [54]:
tracks_df.sample(20)

,track_id,track_title,valid_genre
6085,11377,Franz Schubert - Fantasy in C major D.760 (op....,['5']
3182,5289,Don't want to be your man,['12']
27962,43241,Terrakota live at Sacred Fire Oba Train,"['2', '79']"
42977,64953,Hot Shower,"['5', '15']"
69414,108231,drover,"['15', '38']"
80464,122659,One Last Sunset,"['18', '659', '1235']"
45881,68903,untitled 03,['12']
63611,97530,The Life of Repton,"['15', '42', '400']"
55819,84438,Pajaro Loco,"['12', '66', '1235']"
52031,79412,Transceiver,"['1', '30', '38', '41', '247']"


In [55]:
tracks_df.valid_genre.values

array(["['4', '15', '182']", "['15', '32', '184']", "['12', '25']", ...,
       "['1', '38', '41', '186', '188']", "['12', '27', '38']",
       "['42', '236']"], dtype=object)

In [56]:
tracks_df.track_title

71297            Cardboard Fox
62529                   Doomed
31176         Pass me the fire
1570           Spectacle Bears
25823            Sixteen Words
                 ...          
17082                Amokdrang
81083                      сон
21532      Tantically Ljove IV
75304    Hey Bird Trash Double
70076       Mellow Scabies 0.5
Name: track_title, Length: 104186, dtype: object

In [57]:
## Get complete genre structure
def get_all_structure(estrutura,df_genres):
    ## Get structure from df_genres
    def get_all_structure_from_df(estrutura,df_genres,structure=[]):
        if estrutura == 0:
            return structure
        else:
            structure.append(estrutura)
            get_all_structure_from_df(df_genres[df_genres["genre_id"]==int(estrutura)].parent.values[0],df_genres,structure)
            return structure
    
    return get_all_structure_from_df(estrutura,df_genres,structure=[])
    

In [58]:
# tracks_df['valid_genre'] = tracks_df.track_genres.apply(lambda x: x.strip('][').split(', ') if x != '[]' else None)
tracks_df['valid_genre'] = tracks_df.valid_genre.apply(lambda x: ast.literal_eval(x))

In [59]:
tracks_df['valid_genre']

71297                                         [4, 15, 182]
62529                                        [15, 32, 184]
31176                                             [12, 25]
1570                                                  [12]
25823                                        [10, 66, 362]
                               ...                        
17082                                                 [25]
81083    [1, 18, 32, 33, 38, 74, 94, 125, 247, 250, 514...
21532                                [1, 38, 41, 186, 188]
75304                                         [12, 27, 38]
70076                                            [42, 236]
Name: valid_genre, Length: 104186, dtype: object

In [60]:
tracks_df['last_genre_id'] = tracks_df.valid_genre.apply(lambda x:x[-1] if x != None else None)

In [61]:
tracks_df.sample(20)

,track_id,track_title,valid_genre,last_genre_id
40429,61535,Ociel,[21],21
42191,63998,Brute (Intro),[10],10
72035,111759,Mapper,"[15, 38, 236]",236
31078,48141,Stuck With Me,"[12, 103, 111]",111
86122,130376,Disturbo?,"[42, 47]",47
29247,45115,Dawn at New Hope 2,[30],30
84520,128222,Lunar Flag Assembly,"[74, 250]",250
37080,57094,Honeys Pie,"[15, 811]",811
100539,149475,Improv2@halas.am,"[1, 38, 250]",250
25126,39539,Effects of Elevation (Citizen,"[4, 15, 542]",542


In [62]:
# tracks_df.dropna(inplace=True)

In [63]:
tracks_df

,track_id,track_title,valid_genre,last_genre_id
71297,110788,Cardboard Fox,"[4, 15, 182]",182
62529,95887,Doomed,"[15, 32, 184]",184
31176,48268,Pass me the fire,"[12, 25]",25
1570,1957,Spectacle Bears,[12],12
25823,40488,Sixteen Words,"[10, 66, 362]",362
...,...,...,...,...
17082,28060,Amokdrang,[25],25
81083,123533,сон,"[1, 18, 32, 33, 38, 74, 94, 125, 247, 250, 514...",1235
21532,34452,Tantically Ljove IV,"[1, 38, 41, 186, 188]",188
75304,115900,Hey Bird Trash Double,"[12, 27, 38]",38


In [64]:
tracks_df['full_genre_id'] = tracks_df.last_genre_id.apply(lambda x: get_all_structure(x,genres_df)[::-1])

In [65]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[9, 651, 493]       4
[2, 86, 173]        4
[20, 65, 189]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [66]:
tracks_df.columns

Index(['track_id', 'track_title', 'valid_genre', 'last_genre_id',
       'full_genre_id'],
      dtype='object')

In [67]:
tracks_df = tracks_df[['track_id','full_genre_id']]

In [68]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[9, 651, 493]       4
[2, 86, 173]        4
[20, 65, 189]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [69]:
tracks_df.full_genre_id.info

<bound method Series.info of 71297    [15, 182]
62529    [15, 184]
31176     [12, 25]
1570          [12]
25823    [10, 362]
           ...    
17082     [12, 25]
81083       [1235]
21532    [20, 188]
75304         [38]
70076    [15, 236]
Name: full_genre_id, Length: 104186, dtype: object>

In [70]:
labels_size = tracks_df.full_genre_id.apply(lambda x: len(x))

In [71]:
labels_size = labels_size.max()

In [72]:
labels_size

5

In [73]:
import pandas as pd
import os

### Parse of label to structure

In [74]:
### Function for parse label to sctructure of hierarhical scheme

def parse_label(label,label_size=5):
    # label = label.split('-')
    # preencher com 0 no caso de haver menos de 5 níveis
    labels = np.zeros(label_size,dtype=int)
    for i, label in enumerate(label):
        if i == 5:
            break
        # Aqui você pode fazer a conversão do label em um índice inteiro usando um dicionário ou outro método
        # Neste exemplo, estou apenas usando a posição da label na lista como índice
        labels[i] = label
    return labels

In [75]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[9, 651, 493]       4
[2, 86, 173]        4
[20, 65, 189]       4
[5, 444]            2
[20, 7]             2
Name: full_genre_id, Length: 159, dtype: int64

In [76]:
parsed_labels = tracks_df.full_genre_id.apply(lambda x: parse_label(x))

In [77]:
def convert_label_to_string(x,level=2):
    return '-'.join([str(value) for value in x[:level]])

In [78]:
tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))
tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))
tracks_df['labels_3'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=3))
tracks_df['labels_4'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=4))
tracks_df['labels_5'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=5))

  0%|          | 0/104186 [00:00<?, ?it/s]

/tmp/ipykernel_134427/310633048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))


  0%|          | 0/104186 [00:00<?, ?it/s]

/tmp/ipykernel_134427/310633048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))


  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

  0%|          | 0/104186 [00:00<?, ?it/s]

In [79]:
tracks_df.labels_1.value_counts()

38      22066
12      21710
15      18493
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [80]:
tracks_df['labels_1'].value_counts()

38      22066
12      21710
15      18493
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [81]:
tracks_df = tracks_df[tracks_df['labels_1'].isin(['3','14'])]

In [82]:
tracks_df.to_csv(os.path.join(train_path,"tracks.csv"),index=False)

In [83]:
tracks_df = pd.read_csv(os.path.join(train_path,"tracks.csv"))

In [84]:
categories_df = pd.DataFrame({'level5':tracks_df.labels_5.unique()})

In [85]:
categories_df['level1'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:1]))
categories_df['level2'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:2]))
categories_df['level3'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:3]))
categories_df['level4'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:4]))

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [86]:
categories_df

,level5,level1,level2,level3,level4
0,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0
1,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0
2,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0
3,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0
4,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0


In [87]:
def get_labels_name(x,genres_df):
    levels = 5
    full_name = []
    last_level = 0
    genre_root = ""
    for genre in x.split('-'):
        genre_df = genres_df[genres_df['genre_id'] == int(genre)]
        if genre_df.empty:
            genre_name = genre_root 
        else:
            genre_name = genre_df.title.values.tolist()[0]
            genre_root = genre_name
        
        full_name.append(genre_name)
    full_name = '>'.join(full_name)
        
    return full_name
    # return genres_df[genres_df['genre_id'] == int(x)].title.values.tolist()[0]

In [88]:
categories_df['level5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x,genres_df))

In [89]:
categories_df

,level5,level1,level2,level3,level4,level5_name
0,14-11-0-0-0,14,14-11,14-11-0,14-11-0-0,Soul-RnB>Disco>Disco>Disco>Disco
1,3-0-0-0-0,3,3-0,3-0-0,3-0-0-0,Blues>Blues>Blues>Blues>Blues
2,14-19-0-0-0,14,14-19,14-19-0,14-19-0-0,Soul-RnB>Funk>Funk>Funk>Funk
3,3-567-0-0-0,3,3-567,3-567-0,3-567-0-0,Blues>Gospel>Gospel>Gospel>Gospel
4,14-0-0-0-0,14,14-0,14-0-0,14-0-0-0,Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB>Soul-RnB


In [90]:
def __create_labels__(categories_df):
    data = {
        "label1": {},
        "label2": {},
        "label3": {},
        "label4": {},
        "label5": {},
        "label1_inverse": [],
        "label2_inverse": [],
        "label3_inverse": [],
        "label4_inverse": [],
        "label5_inverse": [],
        "label1_name": {},
        "label2_name": {},
        "label3_name": {},
        "label4_name": {},
        "label5_name": {},
    }

    for idx, cat in enumerate(set(categories_df.level1.values.tolist())):
        data['label1'][cat] = idx
        data['label1_inverse'].append(cat)
        data['label1_count'] = idx + 1

    for idx, cat in enumerate(set(categories_df.level2.values.tolist())):
        data['label2'][cat] = idx
        data['label2_inverse'].append(cat)
        data['label2_count'] = idx + 1
        
    for idx, cat in enumerate(set(categories_df.level3.values.tolist())):
        data['label3'][cat] = idx
        data['label3_inverse'].append(cat)
        data['label3_count'] = idx + 1

    for idx, cat in enumerate(set(categories_df.level4.values.tolist())):
        data['label4'][cat] = idx
        data['label4_inverse'].append(cat)
        data['label4_count'] = idx + 1
        
    for idx, cat in enumerate(set(categories_df.level5.values.tolist())):
        data['label5'][cat] = idx
        data['label5_inverse'].append(cat)
        data['label5_count'] = idx + 1
        
    for cat5,cat1,cat2,cat3,cat4,name5 in categories_df.values:
        
        name1 = '>'.join(name5.split('>')[:1])
        name2 = '>'.join(name5.split('>')[:2])
        name3 = '>'.join(name5.split('>')[:3])
        name4 = '>'.join(name5.split('>')[:4])
        
        
        data['label1_name'][cat1] = name1
        data['label2_name'][cat2] = name2
        data['label3_name'][cat3] = name3
        data['label4_name'][cat4] = name4
        data['label5_name'][cat5] = name5
        
    return data

In [91]:
with open(categories_labels_path, 'w+') as f:
    f.write(json.dumps(__create_labels__(categories_df)))

In [92]:
labels  = __create_labels__(categories_df)

In [93]:
labels['label3_name']

{'14-11-0': 'Soul-RnB>Disco>Disco',
 '3-0-0': 'Blues>Blues>Blues',
 '14-19-0': 'Soul-RnB>Funk>Funk',
 '3-567-0': 'Blues>Gospel>Gospel',
 '14-0-0': 'Soul-RnB>Soul-RnB>Soul-RnB'}